<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Gemma_7B_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing Necessary Dependencies:**

In [ ]:
!pip install -qU bitsandbytes
!pip install -qU trl
!pip install -qU transformers
!pip install -qU peft
!pip install -qU optimum
!pip install -qU datasets
!pip install -qU accelerate
!pip install -qU nltk
!pip install -qU rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 1.4 MB/s eta 0:00:00
     ━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/731.7 MB 1.0 MB/s eta 0:11:34

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from datasets import load_dataset
dataset = load_dataset("kaifahmad/indian-history-hindi-QA-3.4k",split="train")

In [ ]:
dataset

In [ ]:
def process_data_sample(example):
    # Extract relevant information from the example
    question = example['Question']
    answer = example['Answer']
    # Prepare the processed example for a History Question Answering System
    processed_example = (
        "आप एक इतिहास प्रश्न-उत्तर प्रणाली हैं, जो इतिहास से संबंधित ज्ञान और पूछे जाने वाले प्रश्नों को संग्रहीत करती है। "
        "आप इतिहास से संबंधित तथ्य प्रदान कर सकते हैं, प्रश्नों का उत्तर दे सकते हैं और इतिहास संबंधी मार्गदर्शन प्रदान कर सकते हैं।\n\n"
        f"उपयोगकर्ता प्रश्न:\n{question}\n\n"
        f"उत्तर:\n{answer}\n\n"
    )
    return processed_example

In [ ]:
dataset = dataset.map(lambda example: {'text': process_data_sample(example)}, remove_columns=['Question', 'Answer'])

In [ ]:
for i in range(5):
    print(dataset[i]['text'])

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

In [ ]:
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    BitsAndBytesConfig,
    pipeline,
    logging
)
from sklearn.model_selection import train_test_split
from datasets import Dataset
from huggingface_hub import notebook_login
from google.colab import drive
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

**Model Loading and Quantization:**

In [ ]:
model_name = 'google/gemma-2b'

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True
)

In [ ]:
# Tokenization and Padding
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
# LoRA Config
peft_config = LoraConfig(
    r=64,
    lora_alpha=16,
    lora_dropout=0.1,
    bias='none',
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"]
)

In [ ]:
# Training Arguments
training_arguments = TrainingArguments(
    output_dir='Finetuned-Model',
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim='paged_adamw_32bit',
    learning_rate=2e-4,
    lr_scheduler_type='cosine',
    save_strategy='epoch',
    logging_steps=10,
    save_steps=10,
    num_train_epochs=5,
    max_steps=50,
    fp16=True,
    warmup_ratio=0.05,
    push_to_hub=False,
)

In [ ]:
import datasets
train_dataset = datasets.Dataset.from_dict(train_data)
test_dataset = datasets.Dataset.from_dict(test_data)

# SFTTrainer Arguments
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field='text',
    args=training_arguments,
    tokenizer=tokenizer,
    packing=False,
    max_seq_length=264
)

In [ ]:
trainer.train()

In [ ]:
output_model_dir = "Gemma-7B-finetuned-indian-history"
trainer.model.save_pretrained(output_model_dir)
tokenizer.save_pretrained(output_model_dir)

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [ ]:
finetuned_model = AutoModelForCausalLM.from_pretrained(
    output_model_dir,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True
)

In [ ]:
# Tokenization and Padding
tokenizer = AutoTokenizer.from_pretrained(
    output_model_dir,
    trust_remote_code=True,
)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Function to generate responses
def generate_response(context, max_new_tokens=50):
    input_ids = tokenizer.encode(context, return_tensors="pt")
    output = finetuned_model.generate(input_ids, max_new_tokens=max_new_tokens, num_return_sequences=1, temperature=0.7)
    return tokenizer.decode(output[0], skip_special_tokens=True)


test_samples = test_data['text']
generated_responses = [generate_response(sample) for sample in test_samples]
reference_responses = test_data['text']

In [ ]:
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer

bleu_score = corpus_bleu([[r] for r in reference_responses], generated_responses)
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(generated_responses, reference_responses)

print("BLEU Score:", bleu_score)
print("ROUGE Scores:", rouge_scores)